<a href="https://colab.research.google.com/github/sigvehaug/DSF-DCBP/blob/main/CodingTask1_Sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Science Fundamentals for DCBP, S. Haug, University of Bern 

# Coding Task 1

- Deadline: Send your solution notebook per email to Sigve by 2023-03-22 17:00
- Estimated effort: 2-5 hours (if you don't progress after 2 hours of trying, search for help, ChatGPT, Google, colleagues, course tutor)
- Data on Ilias: CCD-Data with metadata file CCD-Metadata


## Tasks (maximum 12.5 points)

- **CT-1.1** Write a method (function) which removes the header information in the 
datafile and saves that information into a separate textfile. [0.5 points] 
- **CT-1.2** Write a function which reduces the data resolution by merging/averaging columns such that there is only one column per 1 nm. [2.0 points]  
- **CT-1.3** Now generalize the above merging so that it works for any number nm er column. [2.0 points]
- **CT-1.4** Implement the saving of the reduced dataset to a file. Do this (1) by writing an explicit loop (write line by line) and (2) by using pandas methods. Measure the running times of the two approaches. [2.0 points]
- **CT-1.5** Write a function which takes two wavelengths as input and plots the difference of the data at these lengths over time. [2.0 points] 
- **CT-1.6** Let the x-axis be in seconds, label the axes with names and units, label the plot with color and legend, make a title for the plot. Save the plot to a file. [2.0 points]
- **CT-1.7** Plot the full and some reduced datasets in the same figure. Is there any visual difference? [2.0 points]

In [ ]:
# CT-1.1
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Courses/DSF-DCBP/Data-CCD/CCD-Data.csv'

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv(path,delimiter=',')
df.head() 
#df.to_csv('CCD-Data-Clean.csv')

,Unnamed: 0.1,Unnamed: 0,Unnamed: 1,316.6137,316.7492,316.8847,317.0203,317.1558,317.2913,317.4268,...,778.9703,779.089,779.2076,779.3263,779.4449,779.5636,779.6822,779.8008,779.9194,780.0381
0,0,2018-10-25 10:32:04.789000,1540456324789,0.329672,0.329672,0.329672,0.090294,-0.716926,-1.829304,-0.686330,...,-0.038483,-0.039623,-0.036920,-0.037309,-0.037299,-0.037476,-0.036856,-0.036853,-0.039100,-0.035545
1,1,2018-10-25 10:32:04.878000,1540456324878,-0.044412,-0.044412,-0.044412,0.157632,-0.672423,-2.183981,-0.492421,...,-0.039428,-0.038486,-0.037309,-0.036357,-0.037289,-0.038728,-0.037214,-0.036902,-0.040210,-0.038165
2,2,2018-10-25 10:32:04.962000,1540456324962,0.338219,0.338219,0.338219,0.330993,-0.879836,-0.000000,-0.559845,...,-0.037725,-0.037534,-0.036560,-0.036152,-0.036554,-0.036875,-0.037351,-0.036746,-0.038686,-0.036903
3,3,2018-10-25 10:32:05.046000,1540456325046,-0.049682,-0.049682,-0.049682,0.519086,-0.681150,-0.000000,-0.417230,...,-0.037613,-0.037168,-0.036369,-0.034549,-0.036252,-0.037948,-0.036713,-0.035706,-0.038203,-0.036822
4,4,2018-10-25 10:32:05.129000,1540456325129,0.335086,0.335086,0.335086,-0.041393,-0.000000,-1.802774,-0.510417,...,-0.040919,-0.038288,-0.035840,-0.035638,-0.039504,-0.040093,-0.039953,-0.037656,-0.038082,-0.035900


In [ ]:
# CT-1.2
import math
new_df = df.iloc[:,3:-1]
new_columns = {col: math.floor(float(col)) for col in new_df.columns}
new_df = new_df.rename(columns=new_columns)
red_df = new_df.groupby(new_df.columns,axis=1).mean()
red_df.head()

,316,317,318,319,320,321,322,323,324,325,...,770,771,772,773,774,775,776,777,778,779
0,0.329672,-0.166957,-0.064746,0.133130,0.123705,-0.249831,0.271327,-0.093151,-0.123458,-0.026454,...,-0.038470,-0.038555,-0.038021,-0.039290,-0.038654,-0.036496,-0.038835,-0.038145,-0.038333,-0.037679
1,-0.044412,-0.411839,-0.133472,2.032979,-0.387814,-0.031617,0.322955,1.744725,-0.051639,-0.147937,...,-0.038387,-0.038100,-0.038118,-0.038622,-0.038308,-0.036896,-0.039010,-0.038400,-0.037930,-0.037812
2,0.338219,-0.094507,-0.024912,0.061890,-0.226986,0.107871,0.121011,-0.208504,-0.418836,0.005751,...,-0.038472,-0.038063,-0.038303,-0.039200,-0.038397,-0.036932,-0.038245,-0.037664,-0.037514,-0.037057
3,-0.049682,0.027980,-0.008338,-0.088350,0.070348,-0.013674,0.307321,0.159532,-0.047932,0.080549,...,-0.037574,-0.037775,-0.037678,-0.038711,-0.038133,-0.036972,-0.038177,-0.037273,-0.036996,-0.036614
4,0.335086,-0.162378,-0.102281,0.045717,-0.088278,0.042099,0.304675,-0.261077,-0.009424,0.153932,...,-0.038489,-0.038433,-0.038135,-0.038991,-0.038178,-0.037010,-0.038540,-0.038076,-0.037606,-0.038132


In [ ]:
# CT-1.3
import math
nm_n = 100
nm_s = 316
new_df = df.iloc[:,3:-1]
new_columns = {col: math.floor(float(col)) for col in new_df.columns}
new_df = new_df.rename(columns=new_columns)
new_columns = {col: (col-nm_s)//nm_n for col in new_df.columns}
print(new_columns)
new_df = new_df.rename(columns=new_columns)
red_df = new_df.groupby(new_df.columns,axis=1).mean()
red_df.head()

{316: 0, 317: 0, 318: 0, 319: 0, 320: 0, 321: 0, 322: 0, 323: 0, 324: 0, 325: 0, 326: 0, 327: 0, 328: 0, 329: 0, 330: 0, 331: 0, 332: 0, 333: 0, 334: 0, 335: 0, 336: 0, 337: 0, 338: 0, 339: 0, 340: 0, 341: 0, 342: 0, 343: 0, 344: 0, 345: 0, 346: 0, 347: 0, 348: 0, 349: 0, 350: 0, 351: 0, 352: 0, 353: 0, 354: 0, 355: 0, 356: 0, 357: 0, 358: 0, 359: 0, 360: 0, 361: 0, 362: 0, 363: 0, 364: 0, 365: 0, 366: 0, 367: 0, 368: 0, 369: 0, 370: 0, 371: 0, 372: 0, 373: 0, 374: 0, 375: 0, 376: 0, 377: 0, 378: 0, 379: 0, 380: 0, 381: 0, 382: 0, 383: 0, 384: 0, 385: 0, 386: 0, 387: 0, 388: 0, 389: 0, 390: 0, 391: 0, 392: 0, 393: 0, 394: 0, 395: 0, 396: 0, 397: 0, 398: 0, 399: 0, 400: 0, 401: 0, 402: 0, 403: 0, 404: 0, 405: 0, 406: 0, 407: 0, 408: 0, 409: 0, 410: 0, 411: 0, 412: 0, 413: 0, 414: 0, 415: 0, 416: 1, 417: 1, 418: 1, 419: 1, 420: 1, 421: 1, 422: 1, 423: 1, 424: 1, 425: 1, 426: 1, 427: 1, 428: 1, 429: 1, 430: 1, 431: 1, 432: 1, 433: 1, 434: 1, 435: 1, 436: 1, 437: 1, 438: 1, 439: 1, 440: 1,

,0,1,2,3,4
0,0.436909,0.156119,0.022555,-0.044217,-0.041099
1,0.553314,0.157455,0.022532,-0.044217,-0.041228
2,0.490024,0.158368,0.022547,-0.044275,-0.041236
3,0.453112,0.157231,0.023133,-0.043571,-0.040611
4,0.438294,0.156210,0.022703,-0.044028,-0.041000


In [ ]:
new_df.columns

Int64Index([316, 316, 316, 317, 317, 317, 317, 317, 317, 317,
            ...
            778, 778, 779, 779, 779, 779, 779, 779, 779, 779],
           dtype='int64', length=3647)

In [ ]:
# CT-1.4
%%time
red_df.to_csv('CCD-Data-Clean.csv')


CPU times: user 7.29 ms, sys: 0 ns, total: 7.29 ms
Wall time: 7.32 ms


In [ ]:
# CT-1.4
%%time
f_in  = open('CCD-Data-Clean.csv','r')
f_out = open('CCD-Data-CT14.txt','w')
lines = f_in.readlines()
for line in lines:
    f_out.writelines(line)



CPU times: user 6.43 ms, sys: 0 ns, total: 6.43 ms
Wall time: 8.79 ms


KeyError: ignored

In [ ]:
# CT-1.5
import math
import matplotlib.pyplot as plt
def plot_delta(local_df,nm_s, nm_e):
  t = (df.iloc[:,2].astype(int) - df.iloc[0,2].astype(int)).to_numpy()
  plt.plot(t,(df[nm_e]-df[nm_s]).to_numpy())
  return t
  
new_df = df.iloc[:,3:-1]
new_columns = {col: math.floor(float(col)) for col in new_df.columns}
new_df = new_df.rename(columns=new_columns)
red_df = new_df.groupby(new_df.columns,axis=1).mean()
line = plot_delta(red_df,316,350)
print(line)

KeyError: ignored

In [ ]:
#df.head()

In [ ]:
#t1 = df.iloc[0:-2,2].astype(int).to_numpy()
#t2 = df.iloc[1:-1,2].astype(int).to_numpy()
df.iloc[:,2].astype(int) - df.iloc[0,2].astype(int)

                  

0          0
1         89
2        173
3        257
4        340
       ...  
350    29615
351    29699
352    29787
353    29871
354    29955
Name: Unnamed: 1, Length: 355, dtype: int64